<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

- Run the ETL process
- Extract bank and market cap data from the JSON file `bank_market_cap.json`
- Transform the market cap currency using the exchange rate data
- Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [1]:
!mamba install pandas==1.3.3 -y
!mamba install requests==2.26.0 -y


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.2) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['pandas==1.3.3']

[+] 0.0s
[+] 0.1s
pkgs/main/linux-64 ━━━━━━━━━━━━╸━━━━━━━━━━━━   0.0 B /  ??.?MB 

## Imports

Import any additional libraries you may need here.


In [2]:
import glob
import pandas as pd
from datetime import datetime
import json

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section  


In [3]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-07-22 10:56:38--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json.6’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-07-22 10:56:38 (21.1 MB/s) - ‘bank_market_cap_1.json.6’ saved [2815/2815]

--2023-07-22 10:56:39--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%2

## Extract


### JSON Extract Function

This function will extract JSON files.


In [4]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [5]:
columns=['Name','Market Cap (US$ Billion)']

In [6]:
with open('bank_market_cap_1.json', 'r') as file:
        data = json.load(file)
data

{'Name': {'0': 'JPMorgan Chase',
  '1': 'Industrial and Commercial Bank of China',
  '2': 'Bank of America',
  '3': 'Wells Fargo',
  '4': 'China Construction Bank',
  '5': 'HSBC Holdings PLC',
  '6': 'Agricultural Bank of China',
  '7': 'Citigroup Inc.',
  '8': 'Bank of China',
  '9': 'China Merchants Bank',
  '10': 'Royal Bank of Canada',
  '11': 'Banco Santander',
  '12': 'Commonwealth Bank',
  '13': 'Mitsubishi UFJ Financial Group',
  '14': 'Toronto-Dominion Bank',
  '15': 'BNP Paribas',
  '16': 'Goldman Sachs',
  '17': 'Sberbank of Russia',
  '18': 'Morgan Stanley',
  '19': 'U.S. Bancorp',
  '20': 'HDFC Bank',
  '21': 'Itaú Unibanco',
  '22': 'Westpac',
  '23': 'Scotiabank',
  '24': 'ING Group',
  '25': 'UBS',
  '26': 'Charles Schwab',
  '27': 'PNC Financial Services',
  '28': 'Lloyds Banking Group',
  '29': 'Sumitomo Mitsui Financial Group',
  '30': 'Bank of Communications',
  '31': 'Australia and New Zealand Banking Group',
  '32': 'Banco Bradesco',
  '33': 'National Australia Ba

In [7]:
def extract():
    # Write your code here
    extracted_data = pd.DataFrame(columns=columns)

    for jsonfile in glob.glob("bank_market_cap_1*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    return extracted_data
    

<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [8]:
# Write your code here
df =pd.read_csv('exchange_rates.csv',index_col=0)
exchange_rate = df.loc["GBP",'Rates']
exchange_rate


0.7323984208000001

## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1. Changes the `Market Cap (US$ Billion)` column from USD to GBP
2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
3. Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [9]:
def transform(data):
    # Read exchange rates into a separate DataFrame
    exchange_rates = pd.read_csv('exchange_rates.csv', index_col=0)
    
    # Get the USD to GBP exchange rate
    exchange_rate = exchange_rates.loc['GBP', 'Rates']
    
    # Make sure the column exists in your dataframe
   # assert 'Market Cap (US$ Billion)' in data.columns, "Column 'Market Cap (US$ Billion)' does not exist in the dataframe"
    
    # Convert Market Cap from USD to GBP and round to 3 decimal places
    data['Market Cap (US$ Billion)'] = (data['Market Cap (US$ Billion)'] * exchange_rate).round(3)
    
    # Rename the column
    data.rename(columns={'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'}, inplace=True)
    
    return data


## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [21]:
targetfile = 'bank_market_cap_gbp.csv'
def load(targetfile , data_load):
    # Write your code here
    data_load.to_csv(targetfile)

## Logging Function


Write the logging function <code>log</code> to log your data:


In [11]:
logfile = 'exchange_rate_logfile.txt'
def log(message):
    # Write your code here
    timestamp = '%H:%M:%S-%h-%d-%Y'
    now=datetime.now()
    time=now.strftime(timestamp)
    with open("exchange_rate_logfile.txt","a") as f:
        f.write(time + ','+message+ '\n')

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [12]:
# Write your code here
log('ETL Job Started')
log('Extract Phase Started')

### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [13]:
# Call the function here
extracted_data = extract()

# Print the rows here
extracted_data

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
65,Ping An Bank,37.993
66,Standard Chartered,37.319
67,United Overseas Bank,35.128
68,QNB Group,33.560


Log the data as <code>"Extract phase Ended"</code>


In [14]:
# Write your code here
log('Extract phase ended')

### Transform


Log the following  <code>"Transform phase Started"</code>


In [15]:
# Write your code here
log('Transform phase Started')

<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [17]:
# Call the function here
transform_data = transform(extracted_data)
# Print the first 5 rows here
transform_data

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519
...,...,...
65,Ping An Bank,27.826
66,Standard Chartered,27.332
67,United Overseas Bank,25.728
68,QNB Group,24.579


Log your data <code>"Transform phase Ended"</code>


In [ ]:
# Write your code here
log("Transform phase Ended")

### Load


Log the following `"Load phase Started"`.


In [ ]:
# Write your code here
log("Load phase started")

Call the load function


In [22]:
# Write your code here
load(targetfile , transform_data)

Log the following `"Load phase Ended"`.


In [23]:
# Write your code here
log("Load phase Ended")

## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


 Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
